In [3]:
#Import
from github import Github
from pprint import pprint
import requests
import pandas as pd
import time
from urllib.request import urlopen
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#Read data from github

#Archivos json
url="https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/netflix_titles.json"
f=requests.get(url)
data=f.json()
# Convertir el archivo de json en un DataFrame
netflix=pd.DataFrame(data)

#Archivos csv
amazon= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/amazon_prime_titles.csv")
disney= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/disney_plus_titles.csv")
hulu= pd.read_csv("https://raw.githubusercontent.com/mayteet/PI01_DATA05/main/Datasets/hulu_titles.csv") 

#Agregamos la columna de plataform y unimos las tablas
netflix["plataform"]= "netflix"
amazon["plataform"]="amazon"
disney["plataform"]="disney"
hulu["plataform"]="hulu"


#Unimos los dataframes
df=pd.concat([netflix, amazon, disney, hulu]) 

data_all=pd.DataFrame([])
Lista_wrong=["min", "Seasons","Season"]
df_analysis= df[["show_id","rating","plataform"]]

for elemento in Lista_wrong:
    
    # Filtrar el Dataframe por columna rating en el que se encuentre la palabra min, Seasons, Season
    df_analysis_filter= df_analysis.loc[df_analysis["rating"].str.contains(elemento, na=False)]
    data_all=pd.concat([data_all,df_analysis_filter])
    
data_all.rename(columns={'rating':"duration"}, inplace=True)


#Sacamos las variables del dataframe principal
df_analysis_2= df[["show_id","duration","plataform"]]
df_analysis_2=df_analysis_2.dropna(subset="duration")

#Juntamos las columnas
data_all=pd.concat([data_all, df_analysis_2])
data_all

#Limpieza de datos
data_all.rename(columns={'duration':"duration_2"}, inplace=True)


data_all["duration_2"]=data_all["duration_2"].str.replace(" min","")
data_all["duration_2"]=data_all["duration_2"].str.replace(" Seasons","")
data_all["duration_2"]=data_all["duration_2"].str.replace(" Season","")

#Juntamos todo en la tabla principal
df_final= pd.merge(df,data_all, on=["show_id","plataform"],how="left")

df_final

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",netflix,90
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",netflix,2
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,netflix,1
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",netflix,1
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,netflix,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23013,s3069,TV Show,Star Trek: The Original Series,NaN,NaN,United States,NaN,1966,TV-PG,3 Seasons,"Action, Adventure, Classics",The 23rd century adventures of Captain James T...,hulu,3
23014,s3070,TV Show,Star Trek: Voyager,NaN,NaN,United States,NaN,1997,TV-PG,7 Seasons,"Action, Adventure, Science Fiction",Catapulted into the distant sector of the gala...,hulu,7
23015,s3071,TV Show,The Fades,NaN,NaN,United Kingdom,NaN,2011,TV-14,1 Season,"Horror, International, Science Fiction",Seventeen-year-old Paul is haunted by apocalyp...,hulu,1
23016,s3072,TV Show,The Twilight Zone,NaN,NaN,United States,NaN,1959,TV-PG,5 Seasons,"Classics, Science Fiction, Thriller",Rod Serling's seminal anthology series focused...,hulu,5


In [ ]:
#Obtener lista de duplicados
df_duplicados=df_final[df_final[["title","type"]].duplicated()]
list_duplicados=df_duplicados["title"].unique()
list_duplicados



#Crear un dataframe donde almacenare la información de las peliculas de las cuales he completado la informacion
data_total=pd.DataFrame([])


for elemento in list_duplicados:
    
    #Filtrar dataframe y ordener por categoría de country para cada pelicula
    df_prueba_fill=df_final[df_final["title"]==elemento].sort_values("cast", ascending= True)
    df_prueba_fill["cast_2"]=df_prueba_fill["cast"].fillna(method="ffill")
    data_total= pd.concat([data_total,df_prueba_fill])

data_total


In [2]:
#Obtener lista de duplicados
df_duplicados=df_final[df_final[["title","type"]].duplicated()]
list_duplicados=df_duplicados["title"].unique()
list_duplicados



#Crear un dataframe donde almacenare la información de las peliculas de las cuales he completado la informacion
data_total=pd.DataFrame([])

#Tener una lista de análisis más reducida
df_cast=df_final[["show_id","type", "plataform","title","cast"]]



for elemento in list_duplicados:
    
    #Filtrar dataframe y ordener por categoría de country para cada pelicula
    df_prueba_fill=df_cast[df_cast["title"]==elemento].sort_values("cast", ascending= True)
    df_prueba_fill["cast_2"]=df_prueba_fill["cast"].fillna(method="ffill")
    data_total= pd.concat([data_total,df_prueba_fill])

data_total


#Sacamos las variables del dataframe principal
df_cast_prin= df[["show_id","type","plataform","title","cast"]]
df_cast_prin=df_cast_prin.dropna(subset="cast")
df_cast_prin.rename(columns={"cast": "cast_2"}, inplace=True)

#Juntamos las columnas
data_total_cast=pd.concat([data_total, df_cast_prin])
data_total_cast.info()



NameError: name 'df_final' is not defined

In [17]:
data_total[data_total["title"]=="Arrested Development"]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,plataform,duration_2,cast_2
4845,s4846,TV Show,Arrested Development,None,"Jason Bateman, Portia de Rossi, Will Arnett, M...",United States,"May 29, 2018",2019,TV-MA,5 Seasons,TV Comedies,It's the Emmy-winning story of a wealthy famil...,netflix,5,"Jason Bateman, Portia de Rossi, Will Arnett, M..."
22991,s3047,TV Show,Arrested Development,NaN,NaN,United States,NaN,2003,TV-14,3 Seasons,"Comedy, Sitcom","In this critically acclaimed series, Michael B...",hulu,3,"Jason Bateman, Portia de Rossi, Will Arnett, M..."
